# !!!IMPORTANT SETUP!!!

## from your GoogleDrive:
1.   create a folder ProjectAML in /content/drive/MyDrive
2.   upload in ProjectAML (the modified version of) python code @ https://github.com/shyam671/AnomalySegmentation_CourseProjectBaseCode
3. create an empty folder /ProjectAML/datasets/Validation_Dataset
4.  import manually all the test dataset in /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset from https://drive.google.com/file/d/1r2eFANvSlcUjxcerjC8l6dRa0slowMpx/view, folder must contain subfolder of the zip (FS_LostFound_full, fs_static, RoadAnomaly, RoadAnomaly21, RoadObstacles)
5. create an empty folder /ProjectAML/datasets/Train_Dataset

## NOTE: training use a high capacity RAM -> Colab Pro, but you can skip since exist a pre-trained model

# **Install required modules**

In [1]:
#do once
!pip3 install 'torch'
!pip3 install 'torchvision'
#optimized version of Pillow
!pip3 install 'Pillow-SIMD'
!pip3 install 'numpy'
!pip3 install 'matplotlib'
#required to work, don't use --visualize
!pip3 install 'visdom'
#script to download train dataset
!pip3 install 'cityscapesscripts'
!pip3 install 'ood-metrics'
#restart after import

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408200 sha256=13704c32d7212c9786763fa01988d877abd500822046e0eba15c8e55a9442f11
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.6/473.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.8 MB/s eta 0:00:00
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26306 sha256=1a29f422c2649aa611a13211c4dbcba30ca866ccff49ab7c1c8e08ff75cc3503
  Stored in directory: /root/.cache/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098b

# **Mount drive**

In [3]:
import torch
#use GPU if available
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #'cpu' # 'cuda' or 'cpu'
print(DEVICE)

from google.colab import drive
drive.mount('/content/drive')


cuda:0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Unmount drive**

In [4]:
#synchronize your google drive after any modification to ProjectAML folder
drive.flush_and_unmount() #synch
drive.mount('/content/drive') #remount

Mounted at /content/drive


In [ ]:
!unzip -q /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/Validation_Dataset.zip \
 -d /content/drive/MyDrive/ProjectAML/datasets/

# **Import and reorganize cityscapes dataset**
### necessary to run IoU evaluation on val set

In [ ]:
#do once
import os
#switch to cpu for download
DEVICE = torch.device("cpu")

!rm -r /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset
!mkdir /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset

#register to https://www.cityscapes-dataset.com and login here
#241MB
!csDownload gtFine_trainvaltest.zip -d /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset
#register to https://www.cityscapes-dataset.com and login here or use saved credentials
#11GB
!csDownload leftImg8bit_trainvaltest.zip -d /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset

#unzip dataset
!unzip -q /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine_trainvaltest.zip \
 -d /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine_trainvaltest
!unzip -q /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit_trainvaltest.zip \
 -d /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit_trainvaltest



Cityscapes username or email address: maxmueh
Cityscapes password: 
Store credentials unencrypted in '/root/.local/share/cityscapesscripts/credentials.json' [y/N]: y


In [ ]:
#make sure that unzip complete successfully before executing this cell

#remove zip
!rm /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine_trainvaltest.zip
!rm /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit_trainvaltest.zip

In [ ]:

#move only necessary folder in /Train_Dataset
!mv /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine_trainvaltest/gtFine \
  /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine
!mv /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit_trainvaltest/leftImg8bit \
  /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit



In [ ]:
#make sure that mv complete successfully before executing this cell

#remove original folder (with README and other stuff)
!rm -r /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine_trainvaltest
!rm -r /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit_trainvaltest

#set required OS env variable to run correctly csCreateTrainIdInstanceImgs
os.environ['CITYSCAPES_DATASET'] = '/content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/'
#create label
!csCreateTrainIdLabelImgs

# return to gpu
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #'cpu' # 'cuda' or 'cpu'

Processing 5000 annotation files
Progress: 100.0 % 

# **START TRAINING**
## skip, trained model already exists

In [ ]:
import os
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/train"
os.chdir(folder_path)

!rm -r /content/drive/MyDrive/ProjectAML/save
!mkdir /content/drive/MyDrive/ProjectAML/save

%run main.py --savedir erfnet_train_cityscape \
 --datadir /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset \
 --decoder --pretrainedEncoder /content/drive/MyDrive/ProjectAML/trained_models/erfnet_encoder_pretrained.pth.tar

#restart 2 time (frist block on visdom)

#python -m visdom.server -port 8097 # run on local pc-> see result on localhost

========== DECODER TRAINING ===========
Loading encoder pretrained in imagenet
/content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit/train
/content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit/val
<class '__main__.CrossEntropyLoss2d'>
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  0.0005029990024359241


KeyboardInterrupt: 

# In Colab training can stop: restart with following code

In [ ]:
import os
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/train"
os.chdir(folder_path)


%run main.py --savedir erfnet_train_cityscape \
 --datadir /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset \
 --decoder --pretrainedEncoder /content/drive/MyDrive/ProjectAML/trained_models/erfnet_encoder_pretrained.pth.tar \
 --resume

# Eval IoU on cityscape

In [2]:
import os
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/eval"
os.chdir(folder_path)
#modify parser: parse all the path elsewhere
%run eval_iou.py --datadir '/content/drive/MyDrive/ProjectAML/datasets/Train_Dataset'


Loading model: ../trained_models/erfnet.py
Loading weights: ../trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully
/content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit/val /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine/val


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Exception in thread Thread-5 (attachment_entry):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 237, in listen
    sock, _ = endpoints_listener.accept()
  File "/usr/lib/python3.10/socket.py", line 293, in accept
    fd, addr = 

0 val/frankfurt/frankfurt_000000_000294_leftImg8bit.png
1 val/frankfurt/frankfurt_000000_000576_leftImg8bit.png
2 val/frankfurt/frankfurt_000000_001016_leftImg8bit.png
3 val/frankfurt/frankfurt_000000_001236_leftImg8bit.png
4 val/frankfurt/frankfurt_000000_001751_leftImg8bit.png
5 val/frankfurt/frankfurt_000000_002196_leftImg8bit.png
6 val/frankfurt/frankfurt_000000_002963_leftImg8bit.png
7 val/frankfurt/frankfurt_000000_003025_leftImg8bit.png
8 val/frankfurt/frankfurt_000000_003357_leftImg8bit.png
9 val/frankfurt/frankfurt_000000_003920_leftImg8bit.png
10 val/frankfurt/frankfurt_000000_004617_leftImg8bit.png
11 val/frankfurt/frankfurt_000000_005543_leftImg8bit.png
12 val/frankfurt/frankfurt_000000_005898_leftImg8bit.png
13 val/frankfurt/frankfurt_000000_006589_leftImg8bit.png
14 val/frankfurt/frankfurt_000000_007365_leftImg8bit.png
15 val/frankfurt/frankfurt_000000_008206_leftImg8bit.png
16 val/frankfurt/frankfurt_000000_008451_leftImg8bit.png
17 val/frankfurt/frankfurt_000000_009291_

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


499 val/munster/munster_000173_000019_leftImg8bit.png
---------------------------------------
Took  267.4530563354492 seconds
Per-Class IoU:
89.30 Road
68.76 sidewalk
80.90 building
23.60 wall
34.23 fence
54.96 pole
41.52 traffic light
61.46 traffic sign
85.60 vegetation
50.53 terrain
68.63 sky
71.30 person
36.43 rider
86.37 car
25.86 truck
46.14 bus
8.92 train
28.99 motorcycle
63.76 bicycle
MEAN IoU:  54.07 %
Loading model: ../trained_models/erfnet.py
Loading weights: ../trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully
/content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit/val /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine/val
0 val/frankfurt/frankfurt_000000_000294_leftImg8bit.png
1 val/frankfurt/frankfurt_000000_000576_leftImg8bit.png
2 val/frankfurt/frankfurt_000000_001016_leftImg8bit.png
3 val/frankfurt/frankfurt_000000_001236_leftImg8bit.png
4 val/frankfurt/frankfurt_000000_001751_leftImg8bit.png
5 val/frankfurt/frankfurt_0

# Run MSP, max logit, max entropy


In [3]:
import os
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/eval"
os.chdir(folder_path)

print("RoadAnomaly21 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png' \
 --anomalyScore 'MSP'
print("RoadObstacle21 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp' \
 --anomalyScore 'MSP'
print("FS_LostFound_full MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png' \
 --anomalyScore 'MSP'
print("fs_static MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg' \
 --anomalyScore 'MSP'
print("RoadAnomaly MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg' \
 --anomalyScore 'MSP'

print("RoadAnomaly21 max logit")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png'
print("RoadObstacle21 max logit")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp'
print("FS_LostFound_full max logit")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png'
print("fs_static max logit")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg'
print("RoadAnomaly max logit")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg'


print("RoadAnomaly21 max entropy")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png' \
--anomalyScore 'maxEntropy'
print("RoadObstacle21 max entropy")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp' \
--anomalyScore 'maxEntropy'
print("FS_LostFound_full max entropy")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png' \
--anomalyScore 'maxEntropy'
print("fs_static max entropy")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg' \
--anomalyScore 'maxEntropy'
print("RoadAnomaly max entropy")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg' \
--anomalyScore 'maxEntropy'

RoadAnomaly21 MSP
Loading model: erfnet.py
Loading weights: ../trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully
(10, 512, 1024)
AUPRC score: 29.03894429644605
FPR@TPR95: 62.51940124230114
RoadObstacle21 MSP
Loading model: erfnet.py
Loading weights: ../trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully
(30, 512, 1024)
AUPRC score: 2.7143910429290163
FPR@TPR95: 65.29901011374024
FS_LostFound_full MSP
Loading model: erfnet.py
Loading weights: ../trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully
(99, 512, 1024)
AUPRC score: 1.7490388138046746
FPR@TPR95: 50.59403252456551
fs_static MSP
Loading model: erfnet.py
Loading weights: ../trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully
(20, 512, 1024)
AUPRC score: 7.472042122551045
FPR@TPR95: 41.83689056458776
RoadAnomaly MSP
Loading model: erfnet.py
Loading weights: ../trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully
(60, 

# Run temperature scaled MSP

In [4]:
import os
from IPython import get_ipython
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/eval"
os.chdir(folder_path)

def run_command(command):
    result = get_ipython().system(command)
    if result != 0:
        print(f"Error executing command: {command}")


for t in [0.5, 0.75, 1.1]:

    # Define the commands in a list
    commands = [
        f'echo Running RoadAnomaly21 temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png --anomalyScore tempMSP --temp {t}",
        f'echo Running RoadObstacle21 temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp --anomalyScore 'tempMSP' --temp {t}",
        f'echo Running FS_LostFound_full temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png --anomalyScore 'tempMSP' --temp {t}",
        f'echo Running fs_static temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg --anomalyScore 'tempMSP' --temp {t}",
        f'echo Running RoadAnomaly temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg --anomalyScore 'tempMSP' --temp {t}"
    ]

    # Execute the commands using subprocess.run
    for command in commands:
        run_command(command)


Running RoadAnomaly21 temp=0.5 MSP
Error executing command: echo Running RoadAnomaly21 temp=0.5 MSP
Loading model: erfnet.py
Loading weights: ../trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully
(10, 512, 1024)
AUPRC score: 27.004669310562978
FPR@TPR95: 62.72222376052643
Error executing command: python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png --anomalyScore tempMSP --temp 0.5
Running RoadObstacle21 temp=0.5 MSP
Error executing command: echo Running RoadObstacle21 temp=0.5 MSP
Loading model: erfnet.py
Loading weights: ../trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully
(30, 512, 1024)
AUPRC score: 2.4233450018232405
FPR@TPR95: 63.2449293767783
Error executing command: python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp --anomalyScore 'tempMSP' --temp 0.5
Running FS_LostFound_full temp=0.5 MSP
Erro

# Grid search of best temperature for MSP

In [ ]:
import os
from IPython import get_ipython
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/eval"
os.chdir(folder_path)

def run_command(command):
    result = get_ipython().system(command)
    if result != 0:
        print(f"Error executing command: {command}")


for i in range(4, -5, -1):
    t = format(f"{10**i}")

    # Define the commands in a list
    commands = [
        f'echo Running RoadAnomaly21 temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png --anomalyScore tempMSP --temp {t}",
        f'echo Running RoadObstacle21 temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp --anomalyScore 'tempMSP' --temp {t}",
        f'echo Running FS_LostFound_full temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png --anomalyScore 'tempMSP' --temp {t}",
        f'echo Running fs_static temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg --anomalyScore 'tempMSP' --temp {t}",
        f'echo Running RoadAnomaly temp={t} MSP',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg --anomalyScore 'tempMSP' --temp {t}"
    ]

    # Execute the commands using subprocess.run
    for command in commands:
        run_command(command)



# Void classifier (background score)

In [ ]:
#redo training (see Slack, add 1.0 to 20th class weight)
import os
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/train"
os.chdir(folder_path)

#before run modify W[19]
%run main.py --savedir erfnet_train_cityscape_20class \
 --datadir /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset \
 --decoder --pretrainedEncoder /content/drive/MyDrive/ProjectAML/trained_models/erfnet_encoder_pretrained.pth.tar



# In Colab training can stop: restart with following code

In [ ]:
import os
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/train"
os.chdir(folder_path)


%run main.py --savedir erfnet_train_cityscape_20class \
 --datadir /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset \
 --decoder --pretrainedEncoder /content/drive/MyDrive/ProjectAML/trained_models/erfnet_encoder_pretrained.pth.tar \
 --resume

In [ ]:
import os
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/eval"
os.chdir(folder_path)



print("RoadAnomaly21 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png' \
 --loadDir '../save/erfnet_train_cityscape_20class/'\
 --anomalyScore 'MSP' --void "yes"

print("RoadObstacle21 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp' \
 --loadDir '../save/erfnet_train_cityscape_20class/'\
 --anomalyScore 'MSP' --void "yes"

print("FS_LostFound_full MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png' \
 --loadDir '../save/erfnet_train_cityscape_20class/'\
 --anomalyScore 'MSP' --void "yes"

print("fs_static MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg' \
 --loadDir '../save/erfnet_train_cityscape_20class/'\
 --anomalyScore 'MSP' --void "yes"

print("RoadAnomaly MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg' \
 --loadDir '../save/erfnet_train_cityscape_20class/'\
 --anomalyScore 'MSP' --void "yes"